In [3]:
%pip install "ultralytics<=8.3.40" supervision roboflow
import ultralytics
ultralytics.checks()

Ultralytics 8.3.40 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (4 CPUs, 31.4 GB RAM, 6362.2/8062.4 GB disk)


In [5]:
from ultralytics import YOLO

model = YOLO("/kaggle/input/appliance-status-detection-model/tensorflow2/default/1/runs/detect/train/weights/best.pt")  # your trained model
class_names = model.names
print(class_names)


{0: 'AC-Off', 1: 'AC-On', 2: 'Fan-Off', 3: 'Fan-On', 4: 'Light-Off', 5: 'Light-On'}


In [4]:
import cv2
from ultralytics import YOLO
import os

# Load the trained YOLO model
model = YOLO("/kaggle/input/appliance-status-detection-model/tensorflow2/default/1/runs/detect/train/weights/best.pt")  # Update with your model weights path
# OR if you want to use a different model that detects fans:
# model = YOLO("yolov8n.pt")  # This is the standard YOLOv8 nano model trained on COCO dataset

# Input and output video paths
input_video = "/kaggle/input/apl-video-for-texting/apl.mp4"
output_dir = "/kaggle/working/processed_video-5"
output_video = os.path.join(output_dir, "output_video.mp4")

# Create output directory
os.makedirs(output_dir, exist_ok=True)

# Open the input video
cap = cv2.VideoCapture(input_video)
if not cap.isOpened():
    print("Error: Could not open video. Check path:", input_video)
    exit()

# Get video properties
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Initialize video writer for output
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(output_video, fourcc, fps, (width, height))

# Process video frame by frame
frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Perform inference with GPU
    results = model(frame, conf=0.25, iou=0.25, device=0)  # Use GPU (device=0)

    # Customize annotations
    for r in results:
        boxes = r.boxes
        for box in boxes:
            cls = int(box.cls)
            conf = float(box.conf)
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            
            # Get class name
            class_name = r.names[cls]
            
            # Only draw boxes for specific classes if needed
            # if class_name not in ['fan', 'tumor', 'necrosis']:
            #     continue
            
            # Set color based on class
            if class_name == 'a':
                color = (0, 0, 255)  
            elif class_name == '':
                color = (0, 255, 0)  
            elif class_name == 'fan':
                color = (255, 0, 0)  # Blue for fan
            else:
                color = (255, 255, 0)  # Yellow for other classes
            
            label = f"{class_name} {conf:.2f}"
            # Draw bounding box and label
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # Write the annotated frame
    out.write(frame)

    frame_count += 1
    print(f"Processed frame {frame_count}/{total_frames}")

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()

print(f"Output video saved at: {output_video}")


0: 640x384 1 Light-On, 52.4ms
Speed: 155.9ms preprocess, 52.4ms inference, 512.5ms postprocess per image at shape (1, 3, 640, 384)
Processed frame 1/1546

0: 640x384 1 Light-On, 10.8ms
Speed: 2.1ms preprocess, 10.8ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 384)
Processed frame 2/1546

0: 640x384 2 Light-Ons, 10.8ms
Speed: 2.3ms preprocess, 10.8ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 384)
Processed frame 3/1546

0: 640x384 2 Light-Ons, 10.8ms
Speed: 1.5ms preprocess, 10.8ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 384)
Processed frame 4/1546

0: 640x384 2 Light-Ons, 10.8ms
Speed: 1.5ms preprocess, 10.8ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 384)
Processed frame 5/1546

0: 640x384 2 Light-Ons, 10.8ms
Speed: 1.7ms preprocess, 10.8ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 384)
Processed frame 6/1546

0: 640x384 2 Light-Ons, 10.8ms
Speed: 1.7ms preprocess, 10.8ms inference, 1.1ms postp

error: OpenCV(4.10.0) /io/opencv/modules/highgui/src/window.cpp:1295: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvDestroyAllWindows'


In [ ]:
import cv2
from ultralytics import YOLO
import os

# Load the trained YOLO11 model
model = YOLO("/kaggle/input/apl-detect-update-model/tensorflow2/default/1/weights/best.pt")  # Update with your model weights path

# Input and output video paths
input_video = "/kaggle/input/apl-video-for-texting/apl.mp4"  # Update with your video path
output_dir = "/kaggle/working/processed_video"
output_video = os.path.join(output_dir, "output_video.mp4")

# Create output directory
os.makedirs(output_dir, exist_ok=True)

# Open the input video
cap = cv2.VideoCapture(input_video)
if not cap.isOpened():
    print("Error: Could not open video. Check path:", input_video)
    exit()

# Get video properties
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Initialize video writer for output
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(output_video, fourcc, fps, (width, height))

# Process video frame by frame
frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Perform inference with GPU
    results = model(frame, conf=0.25, iou=0.25, device=0)  # Use GPU (device=0)

    # Customize annotations for tumors (red) and necrosis (green)
    for r in results:
        boxes = r.boxes
        for box in boxes:
            cls = int(box.cls)
            conf = float(box.conf)
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            # Set color based on class (1: tumor=red, 0: necrosis=green)
            color = (0, 0, 255) if cls == 1 else (0, 255, 0)
            label = f"{r.names[cls]} {conf:.2f}"
            # Draw bounding box and label
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # Write the annotated frame
    out.write(frame)

    frame_count += 1
    print(f"Processed frame {frame_count}/{total_frames}")

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()

print(f"Output video saved at: {output_video}") 


0: 640x384 1 Light-On, 10.1ms
Speed: 1.9ms preprocess, 10.1ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 384)
Processed frame 1/1546

0: 640x384 1 Light-On, 10.1ms
Speed: 1.7ms preprocess, 10.1ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 384)
Processed frame 2/1546

0: 640x384 2 Light-Ons, 8.1ms
Speed: 1.6ms preprocess, 8.1ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 384)
Processed frame 3/1546

0: 640x384 2 Light-Ons, 8.0ms
Speed: 1.4ms preprocess, 8.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)
Processed frame 4/1546

0: 640x384 1 Light-On, 8.1ms
Speed: 1.5ms preprocess, 8.1ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 384)
Processed frame 5/1546

0: 640x384 2 Light-Ons, 9.4ms
Speed: 1.7ms preprocess, 9.4ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)
Processed frame 6/1546

0: 640x384 1 Light-On, 9.5ms
Speed: 1.8ms preprocess, 9.5ms inference, 1.0ms postprocess per image

for fan

In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
import os
from collections import deque

# Load model
model = YOLO("/kaggle/input/apl-detect-update-model/tensorflow2/default/1/weights/best.pt")  # Update path

# Video settings
input_video = "/kaggle/input/apl-video-for-texting/apl.mp4"  # Update path
output_dir = "output_video"
os.makedirs(output_dir, exist_ok=True)
output_video = os.path.join(output_dir, "output.mp4")

cap = cv2.VideoCapture(input_video)
if not cap.isOpened():
    raise FileNotFoundError(f"Video not found: {input_video}")

# Video writer
width, height = int(cap.get(3)), int(cap.get(4))
fps = int(cap.get(5))
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(output_video, fourcc, fps, (width, height))

# Class IDs (update based on your model)
CLASS_IDS = {
    "AC-Off": 0,
    "AC-On": 1,
    "Fan-Off": 2,
    "Fan-On": 3,
    "Light-Off": 4,
    "Light-On": 5
}

# Blur settings (ONLY FOR FAN CLASSES)
FAN_OFF_CLASS = CLASS_IDS["Fan-Off"]
FAN_ON_CLASS = CLASS_IDS["Fan-On"]
BLUR_THRESHOLD = 100  # Tune based on fan speed (lower = more sensitive)
FRAME_HISTORY = 5  # Smoothing window
fan_state_history = deque(maxlen=FRAME_HISTORY)

# Color scheme
COLORS = {
    "AC-Off": (255, 165, 0),   # Orange
    "AC-On": (0, 255, 255),    # Yellow
    "Fan-Off": (0, 255, 0),    # Green
    "Fan-On": (0, 0, 255),     # Red
    "Light-Off": (255, 0, 0),  # Blue
    "Light-On": (255, 0, 255)  # Purple
}

# Process video
frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # YOLO inference
    results = model(frame, conf=0.3, iou=0.45, device=0)  # GPU

    for r in results:
        boxes = r.boxes
        for box in boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            cls = int(box.cls)
            conf = float(box.conf)
            class_name = r.names[cls]
            label = f"{class_name} {conf:.2f}"

            # Apply blur check ONLY for fan classes
            if cls in [FAN_OFF_CLASS, FAN_ON_CLASS]:
                # Crop fan region and check blur
                fan_roi = frame[y1:y2, x1:x2]
                if fan_roi.size > 0:
                    gray = cv2.cvtColor(fan_roi, cv2.COLOR_BGR2GRAY)
                    blur_value = cv2.Laplacian(gray, cv2.CV_64F).var()
                    # Override to Fan-On if blur detected (even if YOLO predicted Off)
                    if blur_value < BLUR_THRESHOLD:
                        cls = FAN_ON_CLASS
                        class_name = "Fan-On"
                    else:
                        cls = FAN_OFF_CLASS
                        class_name = "Fan-Off"
                    fan_state_history.append(cls)
                    # Majority vote for smooth output
                    final_state = max(set(fan_state_history), key=fan_state_history.count)
                    label = f"{class_name} {conf:.2f} (Blur: {blur_value:.1f})"
                else:
                    class_name = "Fan-Off"  # Default if ROI invalid

            # Get color based on class name
            color = COLORS.get(class_name, (255, 255, 255))  # White for unknown

            # Draw bounding box
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

    # Write frame
    out.write(frame)
    frame_count += 1
    print(f"Frame {frame_count}: Processed")

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()
print(f"Output saved: {output_video}")


0: 640x384 1 Light-On, 54.1ms
Speed: 6.7ms preprocess, 54.1ms inference, 622.8ms postprocess per image at shape (1, 3, 640, 384)
Frame 1: Processed

0: 640x384 1 Light-On, 9.7ms
Speed: 2.3ms preprocess, 9.7ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 384)
Frame 2: Processed

0: 640x384 1 Light-On, 8.7ms
Speed: 2.1ms preprocess, 8.7ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 384)
Frame 3: Processed

0: 640x384 1 Light-On, 8.6ms
Speed: 2.1ms preprocess, 8.6ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 384)
Frame 4: Processed

0: 640x384 1 Light-On, 8.2ms
Speed: 2.0ms preprocess, 8.2ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 384)
Frame 5: Processed

0: 640x384 1 Light-On, 8.4ms
Speed: 1.4ms preprocess, 8.4ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)
Frame 6: Processed

0: 640x384 1 Light-On, 8.6ms
Speed: 1.4ms preprocess, 8.6ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)


for image

In [3]:
import cv2
import numpy as np
from ultralytics import YOLO
from collections import deque
import os

# Load YOLO model
model = YOLO("/kaggle/input/appliance-status-detection-model/tensorflow2/default/1/runs/detect/train/weights/best.pt")

# Folder paths
input_folder = "/kaggle/input/video-conert/apl/2.mp4"  # Replace with your folder path
output_folder = "annotated_images"
os.makedirs(output_folder, exist_ok=True)

# Class mappings
CLASS_IDS = {
    "AC-Off": 0,
    "AC-On": 1,
    "Fan-Off": 2,
    "Fan-On": 3,
    "Light-Off": 4,
    "Light-On": 5
}
FAN_OFF_CLASS = CLASS_IDS["Fan-Off"]
FAN_ON_CLASS = CLASS_IDS["Fan-On"]

# Color mapping
COLORS = {
    "AC-Off": (255, 165, 0),
    "AC-On": (0, 255, 255),
    "Fan-Off": (0, 255, 0),
    "Fan-On": (0, 0, 255),
    "Light-Off": (255, 0, 0),
    "Light-On": (255, 0, 255)
}

# Blur detection
BLUR_THRESHOLD = 100
FRAME_HISTORY = 5

# Loop through all image files
for file_name in os.listdir(input_folder):
    if file_name.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp')):
        file_path = os.path.join(input_folder, file_name)
        image = cv2.imread(file_path)
        if image is None:
            print(f"❌ Failed to read: {file_name}")
            continue

        fan_state_history = deque(maxlen=FRAME_HISTORY)

        # YOLO inference
        results = model(image, conf=0.3, iou=0.45, device=0)

        for r in results:
            boxes = r.boxes
            for box in boxes:
                cls = int(box.cls[0])
                conf = float(box.conf[0])
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                class_name = r.names[cls]
                label = f"{class_name} {conf:.2f}"

                # Fan detection and blur check
                if cls in [FAN_OFF_CLASS, FAN_ON_CLASS]:
                    fan_roi = image[y1:y2, x1:x2]
                    if fan_roi.size > 0:
                        gray = cv2.cvtColor(fan_roi, cv2.COLOR_BGR2GRAY)
                        blur_value = cv2.Laplacian(gray, cv2.CV_64F).var()
                        detected_state = FAN_ON_CLASS if blur_value < BLUR_THRESHOLD else FAN_OFF_CLASS
                        fan_state_history.append(detected_state)
                        final_state = max(set(fan_state_history), key=fan_state_history.count)
                        class_name = "Fan-On" if final_state == FAN_ON_CLASS else "Fan-Off"
                        label = f"{class_name} {conf:.2f} (Blur: {blur_value:.1f})"
                    else:
                        class_name = "Fan-Off"
                        label = f"{class_name} {conf:.2f} (Invalid ROI)"

                color = COLORS.get(class_name, (255, 255, 255))
                cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
                cv2.putText(image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

        # Save result
        output_path = os.path.join(output_folder, file_name)
        cv2.imwrite(output_path, image)
        print(f"✅ Processed: {file_name}")

print("🎉 All images processed and saved.")


NotADirectoryError: [Errno 20] Not a directory: '/kaggle/input/video-conert/apl/2.mp4'

In [5]:
import shutil

# Path to the folder you want to zip
folder_to_zip = '/kaggle/working/processed_video-5'

# Output ZIP file name
output_zip_path = "v-3"

# Compress the folder
shutil.make_archive(output_zip_path.replace('.zip', ''), 'zip', folder_to_zip)

print(f"Folder '{folder_to_zip}' successfully compressed into '{output_zip_path}'")

Folder '/kaggle/working/processed_video-5' successfully compressed into 'v-3'
